In [ ]:
__author__ = "Srinath Narayanan"

In [1]:
import gzip
import numpy as np
from collections import defaultdict
# import nltk
# from nltk.sentiment import SentimentAnalyzer
# from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
def readGz(f):
  ret = []
  for l in gzip.open(f):
    ret.append(eval(l))
  return ret

In [3]:
data = readGz("../Data/train.json.gz")

In [51]:
testData = readGz("../Data/test_Helpful.json.gz")

In [9]:
sid = SentimentIntensityAnalyzer()
ss = sid.polarity_scores(train_data[0]['reviewText'])
print train_data[0]['reviewText']
print ss
print ss.values()

These are cute, but they are a little small.  When I put them on, my legs stretch the fabric making the black fade out.
{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'compound': 0.25}
[0.0, 0.913, 0.087, 0.25]


In [126]:
alpha = 0
c = 0
for d in train_data:
    outof = d['helpful']['outOf']
    helpful = d['helpful']['nHelpful']
    if outof == 0:
        ratio = 0
    else:
        c += 1
        ratio = helpful*1.0/outof
    alpha += ratio
alpha = alpha*1.0/c
print alpha

0.783737364232


In [148]:
mae = 0
for i in val_data:
    outof = i['helpful']['outOf']
    helpful = i['helpful']['nHelpful']
    if outof != 0:
        pred = alpha*outof
        mae += np.abs(pred - helpful)
mae = mae*1.0/len(val_data)
print mae

0.258702157422


In [128]:
tX = []
tY = []
for d in train_data:
    rating = d['rating']
    words = len(d['reviewText'].split())
    outof = d['helpful']['outOf']
    helpful = d['helpful']['nHelpful']
    if outof != 0:
        ratio = helpful*1.0/outof
        tY.append(ratio)
        tX.append([1,words,rating])
tX = np.matrix(tX)
tY = np.matrix(tY).T
theta,residuals,rank,s = np.linalg.lstsq(tX, tY)
print theta

[[  5.62218966e-01]
 [  2.11835412e-04]
 [  5.07029148e-02]]


In [131]:
mae = 0
for d in val_data:
    rating = d['rating']
    words = len(d['reviewText'].split())
    outof = d['helpful']['outOf']
    helpful = d['helpful']['nHelpful']
    if outof != 0:
        pred = (theta[0,0] + theta[1,0]*words + theta[2,0]*rating)*outof
        mae += abs(pred - helpful)
mae = mae*1.0/len(val_data)
print mae

0.240245808704


In [132]:
testData = readGz("../Data/test_Helpful.json.gz")
testJson = {}
for l in testData:
    testJson[l['reviewerID']+'-'+l['itemID']] = l

In [133]:
len(ratingDict)

200000

In [134]:
predictions = open("../Predictions/predictions_Helpful.txt", 'w')
for l in open("../Predictions/pairs_Helpful.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i,outOf = l.strip().split('-')
  outOf = int(outOf)
  d = testJson[u+'-'+i]
  rating = d['rating']
  words = len(d['reviewText'].split())
  pred = (theta[0,0] + theta[1,0]*words + theta[2,0]*rating)*outOf
  predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(pred) + '\n')
predictions.close()

In [1]:
print "mae on kaggle : 0.23913\nKaggle Username :vnnsrk"

mae on kaggle : 0.23913
Kaggle Username :vnnsrk


In [5]:
alpha2 = 0
for d in train_data:
    alpha2 += d['rating']
alpha2 = alpha2*1.0/len(train_data)
print alpha2

4.23198


In [6]:
mse = 0
mae = 0
for d in val_data:
    mse += (d['rating'] - alpha2)**2
    mae += np.abs(d['rating'] - alpha2)
mse = mse*1.0/len(val_data)
mae = mae*1.0/len(val_data)
print mse,mae

1.2264713284 0.8894957272


In [136]:
data[0]

{'categories': [['Clothing, Shoes & Jewelry', 'Women'],
  ['Clothing, Shoes & Jewelry',
   'Novelty, Costumes & More',
   'Novelty',
   'Clothing',
   'Women',
   'Leggings']],
 'categoryID': 0,
 'helpful': {'nHelpful': 0, 'outOf': 0},
 'itemID': 'I655355328',
 'rating': 3.0,
 'reviewHash': 'R115160670',
 'reviewText': 'These are cute, but they are a little small.  When I put them on, my legs stretch the fabric making the black fade out.',
 'reviewTime': '05 20, 2014',
 'reviewerID': 'U745881038',
 'summary': 'Cute',
 'unixReviewTime': 1400544000}

In [7]:
def prepareData():
    np.random.shuffle(data)
    train_data = data[0:len(data)-20000]
    val_data = data[len(data)-20000:]
    return train_data,val_data

In [54]:
ratingDict = {}
for d in data:
    u = d['reviewerID']
    i = d['itemID']
    ratingDict[u+'-'+i] = d['rating']

In [9]:
def coordDescent(data,alpha,beta_u,beta_i,reg):
    num = 0
    for d in data:
        u = d['reviewerID']
        i = d['itemID']
        r = d['rating']
        t1 = beta_u[utoi[u]]
        t2 = beta_i[itoi[i]]
        num += (r - (t1+t2))
    alpha = num*1.0/len(data)
    
    for u in userToItem:
        num = 0
        for i in userToItem[u]:
            r = ratingDict[u+'-'+i]
            num += (r - alpha-beta_i[itoi[i]])
        beta_u[utoi[u]] = num*1.0/(reg + len(userToItem[u]))
    
    for i in itemToUser:
        num = 0
        for u in itemToUser[i]:
            r = ratingDict[u+'-'+i]
            num += (r - alpha-beta_u[utoi[u]])
        beta_i[itoi[i]] = num*1.0/(reg + len(itemToUser[i]))
    return (alpha,beta_u,beta_i)
    

In [10]:
def init(data):
    utoi = {}
    itoi = {}
    userToItem = defaultdict(set)
    itemToUser = defaultdict(set)
    uctr = 0
    ictr = 0
    for d in data:
        u = d['reviewerID']
        i = d['itemID']
        userToItem[u].add(i)
        itemToUser[i].add(u)
        if u not in utoi:
            utoi[u] = uctr
            uctr += 1
        if i not in itoi:
            itoi[i] = ictr
            ictr += 1
    return (utoi,itoi,userToItem,itemToUser)

In [115]:
def loss(data,alpha,beta_u,beta_i,reg):
    ret = 0
    for d in data:
        u = d['reviewerID']
        i = d['itemID']
        r = d['rating']
        t = alpha
        if u in utoi :
            t += beta_u[utoi[u]]
        if i in itoi :
            t += beta_i[itoi[i]]
        ret += (t - r)**2
#     s1 = 0
#     for i in beta_u:
#         s1 += (i**2)
#     s2 = 0
#     for i in beta_i:
#         s2 += (i**2)
    s1 = sum(beta_u)**2
    s2 = sum(beta_i)**2 
    ret += reg*(s1+ s2)
    return ret

In [118]:
train_data,val_data = prepareData()
train_data = train_data + val_data
(utoi,itoi,userToItem,itemToUser) = init(train_data)
beta_i = np.random.randint(-1,1,size=(1,len(itoi)))
beta_u = np.random.randint(-1,1,size=(1,len(utoi)))
theta = alpha2
beta_i = list(beta_i[0])
beta_u = list(beta_u[0])
iter = 0
epsilon = 0.00000001
prev_loss = 10000000000
final_loss = 0
threshold = 0
reg = 5.45
while iter < 200:
    (theta,beta_u,beta_i) = coordDescent(train_data,theta,beta_u,beta_i,reg)
    cur_loss = loss(val_data,theta,beta_u,beta_i,reg)
    print iter,cur_loss
    if (prev_loss - cur_loss) <= epsilon:
        if threshold == 3:
            break
        else:
            threshold += 1
            prev_loss = cur_loss
    else:
        threshold = 0
        final_loss = cur_loss
        prev_loss = cur_loss
    iter += 1
print 'Final_loss = ',final_loss
print 'MSE :',calcMSE()

0 79711.8243684
1 61755.845111
2 53498.7268384
3 48412.1801647
4 45348.2123235
5 43574.9562717
6 42583.892198
7 42044.9693568
8 41758.0409664
9 41607.7817149
10 41530.1448423
11 41490.4971326
12 41470.4745223
13 41460.4828738
14 41455.5679003
15 41453.1960159
16 41452.0829298
17 41451.5833832
18 41451.3764211
19 41451.3043998
20 41451.2912182
21 41451.3010688
22 41451.3177796
23 41451.3345577
24 41451.3489543
Final_loss =  41451.2912182
MSE : 0.795445386224


In [119]:
predictions = open("../Predictions/predictions_Rating.txt", 'w')
for l in open("../Predictions/pairs_Rating.txt"):
  if l.startswith("userID"):
    #header
    predictions.write(l)
    continue
  u,i = l.strip().split('-')
  pred = theta
  if u in utoi :
    pred += beta_u[utoi[u]]
  if i in itoi :
    pred += beta_i[itoi[i]]
  if pred > 5:
    pred = 5
  if pred < 1:
    pred = 1
  predictions.write(u + '-' + i + ',' + str(pred) + '\n')
predictions.close()

In [42]:
calcMSE()

0.8533547529344567

In [48]:
all_loss = []
lambdas = [6,6.25,6.5,6.75,7]
for reg in lambdas: 
    train_data,val_data = prepareData()
    #train_data = train_data + val_data
    (utoi,itoi,userToItem,itemToUser) = init(train_data)
    beta_i = np.random.randint(-2,2,size=(1,len(itoi)))
    beta_u = np.random.randint(-2,2,size=(1,len(utoi)))
    theta = alpha2
    beta_i = list(beta_i[0])
    beta_u = list(beta_u[0])
    iter = 0
    threshold = 0
    epsilon = 0.0001
    prev_loss = 10000000000
    final_loss = 0
    while iter < 1000:
        (theta,beta_u,beta_i) = coordDescent(train_data,theta,beta_u,beta_i,reg)
        cur_loss = loss(val_data,theta,beta_u,beta_i,reg)
        if iter%10 == 0:
            print cur_loss,
        if (prev_loss - cur_loss) <= epsilon:
            if threshold == 3:
                break
            else:
                threshold += 1
                prev_loss = cur_loss
        else:
            threshold = 0
            final_loss = cur_loss
            prev_loss = cur_loss
        iter += 1
    all_loss.append(final_loss)
    print '\nlambda = ',reg
    print 'Final_loss = ',final_loss
    print 'MSE : ',calcMSE()

print all_loss

775222285.669 319575.646291 22387.9510894 22315.1181894 22315.0998989 
lambda =  6
Final_loss =  22315.100078
MSE :  1.11575499474
787710889.841 257553.940397 21911.9391839 21866.7206769 
lambda =  6.25
Final_loss =  21866.7124399
MSE :  1.0933356167
792108764.334 208624.857914 22102.7220643 22074.7781028 
lambda =  6.5
Final_loss =  22074.7740897
MSE :  1.10373869807
765239409.223 163496.221526 21811.3790055 21794.5260104 
lambda =  6.75
Final_loss =  21794.5243024
MSE :  1.08972621044
728087449.048 128840.067116 21746.94006 21736.839272 
lambda =  7
Final_loss =  21736.8384989
MSE :  1.08684191806
[22315.10007802447, 21866.712439944295, 22074.77408968422, 21794.524302431855, 21736.83849894401]


In [98]:
def calcMSE():
    mse = 0
    for d in val_data:
        r = d['rating']
        u = d['reviewerID']
        i = d['itemID']
        pred = theta
        if u in utoi :
            pred += beta_u[utoi[u]]
        if i in itoi :
            pred += beta_i[itoi[i]]
        if pred > 5:
            pred = 5
        if pred < 1:
            pred = 1
        mse += (pred-r)**2
    mse = mse*1.0/len(val_data)
    return mse


In [41]:
np.sum(beta_i)

-4.9438559379666458e-07

In [162]:
max(beta_i)

1.246112398671429

In [163]:
bu = np.matrix(beta_u)
minbu = np.argmin(bu)
maxbu = np.argmax(bu)
for u in utoi:
    if(utoi[u] == minbu):
        minu = u
    if(utoi[u] == maxbu):
        maxu = u
print "MAX USER",maxu,beta_u[maxbu]
print "MIN USER",minu,beta_u[minbu]
bi = np.matrix(beta_i)
minbi = np.argmin(bi)
maxbi = np.argmax(bi)
for i in itoi:
    if(itoi[i] == minbi):
        mini = i
    if(itoi[i] == maxbi):
        maxi = i
print "MAX ITEM",maxi,beta_i[maxbi]
print "MIN ITEM",mini,beta_i[minbi]


MAX USER U816486110 1.51376872795
MIN USER U052814411 -2.51254170188
MAX ITEM I558325415 1.24611239867
MIN ITEM I071368828 -2.37318572938
